In [2]:
import base64,os,io,json,sys

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

import spacy
import dill

from io import BytesIO

# from model import *
# import model
from model import *

In [ ]:
model_ckpnt = torch.load("./s11-translation.pt", map_location=torch.device('cpu'))
print(model_ckpnt)

In [3]:
num_words = 11
USE_CUDA = torch.cuda.is_available()

In [4]:
def make_model(src_vocab, tgt_vocab, emb_size=256, hidden_size=512, num_layers=1, dropout=0.1):
    "Helper: Construct a model from hyperparameters."

    attention = BahdanauAttention(hidden_size)

    model = EncoderDecoder(
        Encoder(emb_size, hidden_size, num_layers=num_layers, dropout=dropout),
        Decoder(emb_size, hidden_size, attention, num_layers=num_layers, dropout=dropout),
        nn.Embedding(src_vocab, emb_size),
        nn.Embedding(tgt_vocab, emb_size),
        Generator(hidden_size, tgt_vocab))

    return model.cuda() if USE_CUDA else model

In [5]:
model = make_model(num_words, num_words, emb_size=32, hidden_size=64)
print(model)

EncoderDecoder(
  (encoder): Encoder(
    (rnn): GRU(32, 64, batch_first=True, dropout=0.1, bidirectional=True)
  )
  (decoder): Decoder(
    (attention): BahdanauAttention(
      (key_layer): Linear(in_features=128, out_features=64, bias=False)
      (query_layer): Linear(in_features=64, out_features=64, bias=False)
      (energy_layer): Linear(in_features=64, out_features=1, bias=False)
    )
    (rnn): GRU(160, 64, batch_first=True, dropout=0.1)
    (bridge): Linear(in_features=128, out_features=64, bias=True)
    (dropout_layer): Dropout(p=0.1, inplace=False)
    (pre_output_layer): Linear(in_features=224, out_features=64, bias=False)
  )
  (src_embed): Embedding(11, 32)
  (trg_embed): Embedding(11, 32)
  (generator): Generator(
    (proj): Linear(in_features=64, out_features=11, bias=False)
  )
)


C:\Python38\lib\site-packages\torch\nn\modules\rnn.py:47: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [6]:
model.load_state_dict(torch.load("./s11-translation.pt", map_location=torch.device('cpu'))) 


AttributeError: 'EncoderDecoder' object has no attribute 'copy'